In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
import datetime

current_date = datetime.datetime.now().date()

target_date = datetime.date(2024, 6, 12)

if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [ ]:
import pandas as pd

df = pd.read_csv("data.csv")

In [ ]:
df.head()

# LLMChain

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [20]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [21]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe company that makes a product {product}"
)

In [22]:
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "Queen Size Sheet Set"
chain.run(product=product)

# Simple chain sequence

In [27]:
from langchain.chains import SimpleSequentialChain

In [28]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe company that makes a product {product}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [29]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 word description for the following company: {company_name}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [30]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], verbose=True
)

In [ ]:
overall_simple_chain.run(product)

# Sequential chain

In [33]:
from langchain.chains import SequentialChain

In [34]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following text to english: {review}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="english_review")

In [35]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence: {english_review}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [36]:
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review in: {review}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

In [37]:
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a followup response to the following summary in the specified language. Summary: {summary}, Language: {language}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup")

In [45]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english_review", "summary", "language", "followup"],
    verbose=True,
)

In [ ]:
review = df.Review[5]
overall_chain(review)